In [1]:
import msprime
import numpy as np
import statistics
import math
import allel
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
from scipy import (stats,ndimage)

In [2]:
#number of demes
#d=args.ndemes
d = 36

#dimension of square grid
dim=int(np.sqrt(d))

#define 2d grid to with deme identity
pmat=np.arange(0,d).reshape(dim,dim)

In [3]:
#define function to generate adjacency matrix
#arguments:
#m = migration rate in one direction
#nd = number of demes
def step_mig_mat(m,nd):
    #m is the uni-directional symmetric migration rate
    #NOTE: nd MUST be a squared number
    if(math.sqrt(nd).is_integer()==False):
        raise ValueError("nd must be a squared number (e.g. 4, 9, 16 ...) for the 2D model")
    else:
        nd2=int(math.sqrt(nd))
        #create matrix which will be used to determine which cells are adjacent in 2-dimensions
        #diagonals not considered for now but can be incorporated later if needed
        pmat=np.arange(0,nd).reshape(nd2,nd2)

        #create empty migration matrix to be filled in. This will be the output
        mmat=np.zeros(shape=[nd,nd])

        #go through each cell in pmat and find out which cells are adjacent
        #first define functions to take care of corners and sides
        def contain(ix,max_ix):
            if ix<0:
                return(0)
            if ix>(max_ix-1):
                return(max_ix-1)
            else:
                return(ix)

        for ii in range(nd):
            center_ix=np.where(pmat==ii)
            top_ix=pmat[contain(center_ix[0]-1,nd2),contain(center_ix[1],nd2)]
            bottom_ix=pmat[contain(center_ix[0]+1,nd2),contain(center_ix[1],nd2)]
            left_ix=pmat[contain(center_ix[0],nd2),contain(center_ix[1]-1,nd2)]
            right_ix=pmat[contain(center_ix[0],nd2),contain(center_ix[1]+1,nd2)]

            mmat[ii,top_ix]=mmat[ii,bottom_ix]=mmat[ii,left_ix]=mmat[ii,right_ix]=m
            mmat[top_ix,ii]=mmat[bottom_ix,ii]=mmat[left_ix,ii]=mmat[right_ix,ii]=m

            mmat[ii,ii]=0

    return(mmat)

In [35]:
#generate migration matrix with migration rate provided by user
mig_mat=step_mig_mat(m=0.05,nd=d)

#diploid sample size within each deme
ss=10

mig_mat.shape

(36, 36)

In [38]:
def step_geno(N=1e4,l=1e7,ss_each=2*ss,tmove=1000,mmat=mig_mat):
    #N is the population size for each deme
    #ss_each is the haploid sample size for each deme
    #l is the length of the chromosome
    #tmove is the number of generations past which all lineages are moved into one deme.
    	#The is to reduce computational time when the no. of lineages << ndemes
        #also to mimic migration of an ancient population after which structure is established
        #set to 1000 generations by default

    sample_sizes=[ss_each]*d

    population_configurations = [
    msprime.PopulationConfiguration(sample_size=k)
    for k in sample_sizes]


    if tmove==-9:
         ts=msprime.simulate(Ne=N,
                          population_configurations=population_configurations,
                          migration_matrix=mmat,
                          mutation_rate=1e-08,
                          recombination_rate=1e-08,
                          length=l)
    else:
        # add an extra deme to move lineages to after tmove generations
        population_configurations = population_configurations.append(msprime.PopulationConfiguration(sample_size = 0))

        #no migration to or from this deme until tmove generations
        mmat = np.append(mmat, np.zeros( (1,d) ), axis = 0)
        mmat = np.append(mmat, np.zeros(( (d+1) ,1)), axis = 1)

        #specify demographic event - move all lineages to deme d+1 after tmove generations
        demog=[
            msprime.MassMigration(
                time=tmove,
                source=i,
                destination=d,
                proportion=1.0) for i in range(d)]

        demog.append(#change migration rate among demes to be 0
            msprime.MigrationRateChange(
                time=tmove,
                rate=0))


        ts=msprime.simulate(Ne=N,
                              population_configurations=population_configurations,
                              migration_matrix=mmat,
                              mutation_rate=1e-08,
                              recombination_rate=1e-08,
                              length=l,
                           demographic_events=demog)

    return(ts)

In [39]:
ts=step_geno(N=100,ss_each=2*ss,l=1000,tmove=100,mmat=mig_mat)

ValueError: Either sample_size, population_configurations, samples or from_ts must be specified

In [7]:
sample_sizes=[2*1]*d

population_configurations = [
msprime.PopulationConfiguration(sample_size=k)
for k in sample_sizes]


ts=msprime.simulate(Ne=100,
                          population_configurations=population_configurations,
                          migration_matrix=mig_mat,
                          mutation_rate=1e-08,
                          recombination_rate=1e-08,
                          length=100000,
                   )  

print(ts)

╔════════════════════════╗
║TreeSequence            ║
╠═══════════════╤════════╣
║Trees          │      63║
╟───────────────┼────────╢
║Sequence Length│100000.0║
╟───────────────┼────────╢
║Sample Nodes   │      72║
╟───────────────┼────────╢
║Total Size     │20.6 KiB║
╚═══════════════╧════════╝
╔═══════════╤════╤═════════╤════════════╗
║Table      │Rows│Size     │Has Metadata║
╠═══════════╪════╪═════════╪════════════╣
║Edges      │ 352│  9.6 KiB│          No║
╟───────────┼────┼─────────┼────────────╢
║Individuals│   0│  8 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Migrations │   0│  4 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Mutations  │  67│  1.9 KiB│          No║
╟───────────┼────┼─────────┼────────────╢
║Nodes      │ 189│  4.4 KiB│          No║
╟───────────┼────┼─────────┼────────────╢
║Populations│  36│148 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Provenances│   1│635 Bytes│          No║
╟───────────┼────┼─────────┼───

In [74]:
#generate migration matrix with migration rate provided by user
mmat=step_mig_mat(m=1,nd=d)

sample_sizes=[2*1]*d

population_configurations = [
msprime.PopulationConfiguration(sample_size=k)
for k in sample_sizes]

# add an extra deme to move lineages to after tmove generations
population_configurations.append(msprime.PopulationConfiguration(sample_size = 0))

#no migration to or from this deme until tmove generations
mmat = np.append(mmat, np.zeros( (1,d) ), axis = 0)
mmat = np.append(mmat, np.zeros(( (d+1) ,1)), axis = 1)

#specify demographic event - move all lineages to deme d+1 after tmove generations
demog=[msprime.MassMigration(
                time=100,
                source=i,
                destination=d,
                proportion=1.0) for i in range(d)]

demog.append(#change migration rate among demes to be 0
            msprime.MigrationRateChange(
                time=100,
                rate=0))

#dd = msprime.DemographyDebugger(
#        population_configurations=population_configurations,
#        demographic_events=demog)
#dd.print_history()

ts=msprime.simulate(Ne=10000,
                              population_configurations=population_configurations,
                              migration_matrix=mmat,
                              mutation_rate=1e-08,
                              recombination_rate=1e-08,
                              length=1000000,
                           demographic_events=demog)

ts

In [56]:
population_configurations = [
msprime.PopulationConfiguration(sample_size=k)
for k in sample_sizes]

population_configurations.append(msprime.PopulationConfiguration(sample_size = 0))
population_configurations